# Centralized baseline

Train a simple logistic regression on the full CSV and record AUC/F1.

In [ ]:
# Preprocess data to CSV-only splits
!python -m scripts.preprocess_data --input data/creditcard.csv --clients 3 --out-root data --seed 42 --test-size 0.2

In [ ]:
# Train centralized baseline and compute metrics
!python -m scripts.train_centralized --train data/processed/train.csv --test data/processed/test.csv --label Class --epochs 2 --lr 0.1 --batch-size 512 --reg 0.0